# ========================================================
# Dark URL Detection
# Adrien Manciet - Thibault Sourdeval
# ========================================================

In [ ]:
#Introduction

# =========================
# Partie 1 : Phase d'exploration
# =========================

In [1]:
#Code

# =========================
# Partie 2 : Feature Engineering
# =========================

In [2]:
#Code

# =========================
# Partie 3 : Phase d'apprentissage
# =========================

In [3]:
#Code

# =========================
# Partie 4 : Tuning d'un hyperparamètre
# =========================

In [4]:
#Code

# =========================
# Partie 5 : Conclusions
# =========================

In [5]:
#Code